In [1]:
# Link do wikipedia - https://pt.wikipedia.org/wiki/Lista_de_munic%C3%ADpios_do_Brasil

In [2]:
# Bibliotecas (importar agora, pois eu vou esquecer)
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re

In [38]:
# Abrindo a página do Wikipedia
url = "https://pt.wikipedia.org/wiki/Lista_de_munic%C3%ADpios_do_Brasil"

In [39]:
# Criando o objeto do BeautifulSoup
soup = BeautifulSoup(html, "html.parser")

In [92]:
# Criando o padrão de expressão regular para obter os dados a partir das siglas
expressao_regular = " \([A-Z]{2}\)$"

In [91]:
def retornar_dataframe_cidade_estado(url, expressao_regular):
    html = urlopen(url)
    # Criando o objeto do BeautifulSoup
    soup = BeautifulSoup(html, "html.parser")
    # Criando o obgeto do regex
    padrao_estado = re.compile(expressao_regular)
    
    # Novo dataframe para mostrar a diferença entre os códigos
    retorno = pd.DataFrame(columns=(["cidade", "estado"]))
    lista_linhas = []

    for linha in soup.find_all("li"):
        # Verficando se é o valor das cidades/estados
        if padrao_estado.search(linha.text):
            valores = padrao_estado.split(linha.text)
            valores[1] = linha.text[-3:-1]
            lista_linhas.append({"cidade": valores[0], "estado": valores[1]})

    # Criar o dataframe final
    retorno = pd.concat([pd.DataFrame([valor]) for valor in lista_linhas], ignore_index=True)
    return retorno

In [93]:
res = retornar_dataframe_cidade_estado(url, expressao_regular)

In [94]:
res.head()

,cidade,estado
0,Abadia de Goiás,GO
1,Abadia dos Dourados,MG
2,Abadiânia,GO
3,Abaeté,MG
4,Abaetetuba,PA


In [11]:
# Acessando o SQL Server
import pyodbc

In [24]:
driver = "SQL Server"
server = "DESKTOP-15TQLGD\SQLEXPRESS"
base_dados = "Agiobot"
# Concatenando direto no parâmetro
# conexao = pyodbc.connect("DRIVER=" + driver + "; Server=" + server + "; Database=" + base_dados + "; TrustedConnection=yes")
# Concatenando em uma outra variável
string_conexao = 'DRIVER={0}; Server={1}; Database={2}; TrustedConnection=yes'.format(driver, server, base_dados)
conexao = pyodbc.connect(string_conexao)
cursor = conexao.cursor()

In [68]:
# Testando se conectou
cursor.execute("SELECT * FROM tbl_Cidades")

In [69]:
linha = cursor.fetchone()
res = []
while linha:
    for coluna in linha:
        res.append(coluna)
    print(res)
    res = []
    linha = cursor.fetchone()

In [89]:
# Lista de tuplas sem o Pandas
df1 = pd.DataFrame(columns=(["cidade", "estado"]))
lista_linhas = []

for linha in soup.find_all("li"):
    # Verficando se é o valor das cidades/estados
    if padrao_estado.search(linha.text):
        #valores = re.split(" \(", linha.text.replace(")",""))
        valores = padrao_estado.split(linha.text)
        valores[1] = linha.text[-3:-1]
        lista_linhas.append((valores[0], valores[1]))

In [53]:
# Executando a inserção de uma linha
cursor.execute("INSERT INTO tbl_Cidades(nomeCidade, siglaEstado) VALUES (?,?)", lista_linhas[0])

In [90]:
# Executando a inserção com várias linhas
cursor.executemany("INSERT INTO tbl_Cidades(nomeCidade, siglaEstado) VALUES (?,?)", lista_linhas)

In [83]:
re.split(" \([A-Z]{2}\)$", "Arez (Rio Grande do Norte) (RN)")

['Arez (Rio Grande do Norte)', '']

In [85]:
"Arez (Rio Grande do Norte) (RN)"[-3:-1]

'RN'

In [87]:
padrao_estado.split("Arez (Rio Grande do Norte) (RN)")

['Arez (Rio Grande do Norte)', '']